In [ ]:
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore') 

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -qq '/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/open.zip'

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/prepro_0428_alive_number_train.csv", index_col = 0)
test = pd.read_csv("/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/prepro_0428_alive_number_test.csv", index_col = 0)

In [ ]:
train.isnull().sum()

ID                         0
User-ID                    0
Book-ID                    0
Book-Rating                0
Age                        0
Location                   0
Book-Title               146
Book-Author                0
Year-Of-Publication        0
Publisher                  0
Age_Category               0
Location2                  0
Book-Title_stop_words      0
dtype: int64

In [ ]:
train = train.dropna(subset=['Location2'])

In [ ]:
train.head()

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age_Category,Location2,Book-Title_stop_words
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",road taken,rona jaffe,2001.0,mira,20대,sackville new brunswick canada,road taken
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",macbeth new penguin shakespeare,william shakespeare,1981.0,penguin books,20대,sackville new brunswick canada,macbeth new penguin shakespeare
2,TRAIN_000002,USER_00000,BOOK_086781,0,23.0,"sackville, new brunswick, canada",waverley penguin english library,walter scott,1981.0,penguin books,20대,sackville new brunswick canada,waverley penguin english library
3,TRAIN_000003,USER_00000,BOOK_098622,0,23.0,"sackville, new brunswick, canada",mother earth father sky,sue harrison,1991.0,avon,20대,sackville new brunswick canada,mother earth father sky
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",she who remembers,linda lay shuler,1989.0,signet book,20대,sackville new brunswick canada,she who remembers


In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
import tensorflow as tf
tf.random.set_seed(812)

In [ ]:
train = TabularDataset(train)
test = TabularDataset(test)

label = 'Book-Rating'
eval_metric = 'root_mean_squared_error'
time_limit = 3600*6

In [ ]:
train.info()

<class 'autogluon.core.dataset.TabularDataset'>
Int64Index: 867682 entries, 0 to 871392
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     867682 non-null  object 
 1   User-ID                867682 non-null  object 
 2   Book-ID                867682 non-null  object 
 3   Book-Rating            867682 non-null  int64  
 4   Age                    867682 non-null  float64
 5   Location               867682 non-null  object 
 6   Book-Title             867536 non-null  object 
 7   Book-Author            867682 non-null  object 
 8   Year-Of-Publication    867682 non-null  float64
 9   Publisher              867682 non-null  object 
 10  Age_Category           867682 non-null  object 
 11  Location2              867682 non-null  object 
 12  Book-Title_stop_words  867682 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 92.7+ MB


In [ ]:
train = train.drop(["ID", "Age", "Location", "Book-Title"], axis = 1)
test = test.drop(["ID", "Age", "Location", "Book-Title"], axis = 1)

In [ ]:
train.info()

<class 'autogluon.core.dataset.TabularDataset'>
Int64Index: 867682 entries, 0 to 871392
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   User-ID                867682 non-null  object 
 1   Book-ID                867682 non-null  object 
 2   Book-Rating            867682 non-null  int64  
 3   Book-Author            867682 non-null  object 
 4   Year-Of-Publication    867682 non-null  float64
 5   Publisher              867682 non-null  object 
 6   Age_Category           867682 non-null  object 
 7   Location2              867682 non-null  object 
 8   Book-Title_stop_words  867682 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 66.2+ MB


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
train['Year-Of-Publication'] = train['Year-Of-Publication'].astype(int)
train['Year-Of-Publication'] = train['Year-Of-Publication'].astype('category')

test['Year-Of-Publication'] = test['Year-Of-Publication'].astype(int)
test['Year-Of-Publication'] = test['Year-Of-Publication'].astype('category')

In [ ]:
le = LabelEncoder()
train["Age_Category"] = le.fit_transform(train["Age_Category"])
test["Age_Category"] = le.transform(test["Age_Category"])

In [ ]:
train = train.astype(({"User-ID" : 'category',
                       "Book-ID" : 'category',
                       "Book-Rating" : 'int',
                       "Book-Author" : "category",
                       'Year-Of-Publication' : "category",
                       "Publisher" : "category",
                       "Age_Category" : "category",
                       'Location2' : "category",
                       'Book-Title_stop_words' : 'category'
                       }))

test = test.astype(({"User-ID" : 'category',
                       "Book-ID" : 'category',
                       "Book-Author" : "category",
                       'Year-Of-Publication' : "category",
                       "Publisher" : "category",
                       "Age_Category" : "category",
                     'Location2' : "category",
                     'Book-Title_stop_words' : 'category'

                       }))

In [ ]:
train.info()

<class 'autogluon.core.dataset.TabularDataset'>
Int64Index: 867682 entries, 0 to 871392
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   User-ID                867682 non-null  category
 1   Book-ID                867682 non-null  category
 2   Book-Rating            867682 non-null  int64   
 3   Book-Author            867682 non-null  category
 4   Year-Of-Publication    867682 non-null  category
 5   Publisher              867682 non-null  category
 6   Age_Category           867682 non-null  category
 7   Location2              867682 non-null  category
 8   Book-Title_stop_words  867682 non-null  category
dtypes: category(8), int64(1)
memory usage: 57.7 MB


In [ ]:
train.head()

,User-ID,Book-ID,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Age_Category,Location2,Book-Title_stop_words
0,USER_00000,BOOK_044368,8,rona jaffe,2001,mira,2,sackville new brunswick canada,road taken
1,USER_00000,BOOK_081205,8,william shakespeare,1981,penguin books,2,sackville new brunswick canada,macbeth new penguin shakespeare
2,USER_00000,BOOK_086781,0,walter scott,1981,penguin books,2,sackville new brunswick canada,waverley penguin english library
3,USER_00000,BOOK_098622,0,sue harrison,1991,avon,2,sackville new brunswick canada,mother earth father sky
4,USER_00000,BOOK_180810,8,linda lay shuler,1989,signet book,2,sackville new brunswick canada,she who remembers


In [ ]:
excluded_model_types = ["KNN", "XT", "NN", "FASTAI"]
# 4부터 과적합 발생
predictor = TabularPredictor(
    label=label, problem_type='regression', eval_metric=eval_metric, path="/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/model_0513"
).fit(train, 
      presets='best_quality', 
      num_stack_levels=2,
      excluded_model_types = excluded_model_types,
      time_limit=time_limit, num_gpus=0)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 50400s
AutoGluon will save models to "/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/model_0513/"
AutoGluon Version:  0.7.0
Python Version:     3.10.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Train Data Rows:    867682
Train Data Columns: 8
Label Column: Book-Rating
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    25049.66 MB
	Train Data (Original)  Memory Usage: 70.46 MB (0.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...

In [ ]:
print(predictor.leaderboard())

                     model  score_val  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L4  -3.200169   14416.966086  19542.450477                0.015755          10.711119            4       True         15
1        LightGBMXT_BAG_L3  -3.204593   14357.442988  17343.472193               63.222117         174.430337            3       True         10
2          CatBoost_BAG_L3  -3.206691   14301.835708  17606.779410                7.614837         437.737554            3       True         13
3          LightGBM_BAG_L3  -3.207531   14314.743960  17286.342362               20.523089         117.300506            3       True         11
4   RandomForestMSE_BAG_L3  -3.208315   14325.590288  18802.270961               31.369417        1633.229105            3       True         12
5          CatBoost_BAG_L2  -3.212806   14225.199630  15475.784086               42.148441        7660.895932            2       T

In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test, model=model_to_use)
print(model_pred[:20])

0     5.634284
1     6.025013
2     5.742294
3     6.016082
4     4.904624
5     4.916741
6     5.288690
7     3.230866
8     3.710062
9     3.621282
10    4.564205
11    5.494278
12    3.700205
13    4.650793
14    5.360093
15    5.194953
16    5.750336
17    3.463907
18    2.540449
19    8.468580
Name: Book-Rating, dtype: float32


In [ ]:
submission = pd.read_csv('/content/sample_submission.csv')

submission["Book-Rating"] = model_pred
submission.to_csv('/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/sub_autogluon_0505_num2_WeightedEnsemble_L4_1st.csv', index=False, encoding="utf-8")

In [ ]:
best_result = pd.read_csv("/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/sub_autogluon_0505_num2_WeightedEnsemble_L4_1st.csv")

In [ ]:
best_result.isnull().sum()

ID               0
Book-Rating    513
dtype: int64

In [ ]:
na_to_zero = best_result.fillna(0)

In [ ]:
na_to_zero.isnull().sum()

ID             0
Book-Rating    0
dtype: int64

In [ ]:
na_to_zero.to_csv('/content/drive/MyDrive/Dacon/제2회 코스포 x 데이콘 도서 추천 알고리즘 AI경진대회(채용 연계형)/sub_autogluon_0505_num2_WeightedEnsemble_L4_1st_fillna.csv', index=False, encoding="utf-8")